In [12]:
import tensorflow as tf
from keras.applications import ResNet152V2 
#import keras_resnet.models
from keras.models import Sequential,Model 
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import keras

In [13]:
# NasNet is designed to work with images of dim 224,224
img_rows,img_cols = 224,224

ResNet152V2 = ResNet152V2(weights=None,include_top=False,input_shape=(img_rows,img_cols,3))

for layer in ResNet152V2.layers:
    layer.trainable = True
    
for (i,layer) in enumerate(ResNet152V2.layers):
    print(str(i),layer.__class__.__name__,layer.trainable)

0 InputLayer True
1 ZeroPadding2D True
2 Conv2D True
3 ZeroPadding2D True
4 MaxPooling2D True
5 BatchNormalization True
6 Activation True
7 Conv2D True
8 BatchNormalization True
9 Activation True
10 ZeroPadding2D True
11 Conv2D True
12 BatchNormalization True
13 Activation True
14 Conv2D True
15 Conv2D True
16 Add True
17 BatchNormalization True
18 Activation True
19 Conv2D True
20 BatchNormalization True
21 Activation True
22 ZeroPadding2D True
23 Conv2D True
24 BatchNormalization True
25 Activation True
26 Conv2D True
27 Add True
28 BatchNormalization True
29 Activation True
30 Conv2D True
31 BatchNormalization True
32 Activation True
33 ZeroPadding2D True
34 Conv2D True
35 BatchNormalization True
36 Activation True
37 MaxPooling2D True
38 Conv2D True
39 Add True
40 BatchNormalization True
41 Activation True
42 Conv2D True
43 BatchNormalization True
44 Activation True
45 ZeroPadding2D True
46 Conv2D True
47 BatchNormalization True
48 Activation True
49 Conv2D True
50 Conv2D True
51 A

In [15]:
def addTopModelNASNetMobile(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(512,activation='relu')(top_model)
    
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

num_classes = 3  # ['COVID19','NORMAL','Viral Pneumonia']

FC_Head = addTopModelNASNetMobile(ResNet152V2, num_classes)

model = Model(inputs = ResNet152V2.input, outputs = FC_Head)

print(model.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_______________________________________________________________________________________

In [16]:
train_data_dir = r'D:\Dev65\python_workspace2\test\collection-re2\stacking'
validation_data_dir = r'D:\Dev65\python_workspace2\test\collection-re2\fuzzyColor'

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=15,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )
print(type(train_datagen))
validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 16

<class 'tensorflow.python.keras.preprocessing.image.ImageDataGenerator'>


In [18]:

train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

# 경로를 통해 labeling 한 데이터 셋을 만드는 것
validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')


Found 3036 images belonging to 3 classes.
Found 3036 images belonging to 3 classes.


In [19]:
from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [20]:


checkpoint = ModelCheckpoint(
                             'ResNet152V2ver1.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

# earlystop = EarlyStopping(
#                           monitor='val_loss',
#                           min_delta=0,
#                           patience=10,
#                           verbose=1,restore_best_weights=True)

# learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
#                                             patience=5, 
#                                             verbose=1, 
#                                             factor=0.2, 
#                                             min_lr=0.0001)

#callbacks = [earlystop,checkpoint,learning_rate_reduction]
callbacks = [checkpoint]


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

nb_train_samples = 3036 
nb_validation_samples = 3036 

epochs = 50

history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,     
            epochs=epochs,
            callbacks=callbacks,
            #validation_split=0.2
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size
)

print("끝났다!")

model.save('ResNet152V2ver1_complete.h5')
model.save_weights('ResNet152V2ver1_Weight.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
189/189 [==============================] - ETA: 0s - loss: 0.7132 - accuracy: 0.7146
Epoch 00001: val_loss improved from inf to 2.53611, saving model to ResNet152V2ver1.h5
189/189 [==============================] - 1480s 8s/step - loss: 0.7132 - accuracy: 0.7146 - val_loss: 2.5361 - val_accuracy: 0.4306
Epoch 2/50
189/189 [==============================] - ETA: 0s - loss: 0.4965 - accuracy: 0.8202
Epoch 00002: val_loss improved from 2.53611 to 1.27835, saving model to ResNet152V2ver1.h5
189/189 [==============================] - 1494s 8s/step - loss: 0.4965 - accuracy: 0.8202 - val_loss: 1.2783 - val_accuracy: 0.4878
Epoch 3/50
189/189 [==============================] - ETA: 0s - loss: 0.3967 - accuracy: 0.8560
Epoch 00003: val_loss improved from 1.27835 to 0.60491, saving model to ResNet152V2ver1.h5
189/189 [==============================] - 1484s 8s/step - loss: 0.3967 - accuracy: 0.8560 - val_loss

Epoch 29/50
189/189 [==============================] - ETA: 0s - loss: 0.1820 - accuracy: 0.9368
Epoch 00029: val_loss did not improve from 0.34440
189/189 [==============================] - 1476s 8s/step - loss: 0.1820 - accuracy: 0.9368 - val_loss: 2.9108 - val_accuracy: 0.5324
Epoch 30/50
189/189 [==============================] - ETA: 0s - loss: 0.1842 - accuracy: 0.9354
Epoch 00030: val_loss did not improve from 0.34440
189/189 [==============================] - 1476s 8s/step - loss: 0.1842 - accuracy: 0.9354 - val_loss: 1.1992 - val_accuracy: 0.5129
Epoch 31/50
189/189 [==============================] - ETA: 0s - loss: 0.1876 - accuracy: 0.9321
Epoch 00031: val_loss did not improve from 0.34440
189/189 [==============================] - 1479s 8s/step - loss: 0.1876 - accuracy: 0.9321 - val_loss: 0.6091 - val_accuracy: 0.8237
Epoch 32/50
189/189 [==============================] - ETA: 0s - loss: 0.1679 - accuracy: 0.9341
Epoch 00032: val_loss did not improve from 0.34440
189/189 [